In [1]:
import numpy as np
import time
from IPython.display import clear_output
import os
from skimage import io
import matplotlib.pyplot as plt
import re

In [2]:
path = 'images/'
images = os.listdir(path)
monster_pics = [i for i in images if 'monster' in i]
character_pics = [i for i in images if 'character' in i]
potion_pics = [i for i in images if 'potion' in i]

## Global variables

In [13]:
monster_names = ['Gorzor', 'Roxxor', 'Morgor', 'Ragnor', 'Grasiv', 'Praxxis', 'Eragor']
monster_attr = ['health', 'endurance', 'damage']

potions = {
    # red-damage, green-endurance, blue-health, yellow-luck
    'damage': {'colour': 'red', 'health': 2, 'luck': 1, 'damage': 8, 'endurance': 1},
    'endurance': {'colour': 'green', 'health': 1, 'luck': 1, 'damage': 1, 'endurance': 9},
    'health': {'colour': 'blue', 'health': 9, 'luck': 1, 'damage': 1, 'endurance': 1},
    'luck': {'colour': 'yellow', 'health': 1, 'luck': 7, 'damage': 2, 'endurance': 2}
}

help_string = """
            • You can either fight a monster, run from it, or try to befriend it.
            • Fighting will inflict damage on both of you till one is dead. 
            • Damage inflicted is proportional to your current health.
            • Running will lower your endurance. If your endurance is lower than the monster, you'll be forced to fight.
            • There's usually a 10% chance of a mosnter being friendly, which goes up with your luck.
            • If a monster is friendly, you'll be allowed to pass, otherwise you'll take extra damage.
            • Potions replenish your attributes. Each colour targets specific attribute as follows:
                ⁃ Red: damage, 
                ⁃ Green: endurance
                ⁃ Blue: health
                ⁃ Yellow-luck
            • if you are unlucky, a potion may turn out to be a trap and it will drain your attributes instead.
            """

## Helper functions

In [12]:
def prints(*args):
    args = [str(i) for i in args]
    string = " ".join(args)
    m = re.findall(r'\d+\.\d+', string)
    for i in m:
        string = re.sub(i, str(round(float(i),2)), string)
    print(string)
    time.sleep(1)
    
def inputs(player, string):
    while True:
        x = input(string)
        if any(s in x for s in ["stop", "quit", "leave"]):
            quit(player)
            return player
        elif any(s in x for s in ["info", "status", "health"]):
            print("Your current status is:")
            player.info()
        elif any(s in x for s in ["tips", "tip", "help", "rules", "rule"]):
            print(help_string)
        else:
            break
    return x
    
def rand():
    rand = round(np.random.random() + 0.5,2)
    return rand

def quit(player):
    player.quit = True
    print("You quit with a score of " + str(round(player.score,2)))
    return player

def check_trap(luck):
    chance = np.random.randint(100)
    trap = True if chance > luck else False
    return trap

## Character definition

In [5]:
class character:
    def __init__(self, name):
        self.name = name
        self.luck = 50
        self.health = 100
        self.init_health = 100
        self.damage = 12
        self.endurance = 15
        self.score = 0
        self.xp = 0
        self.skill = 20
        self.quit = False
        prints("Hello", self.name)
        
    def fight(self, monster):
        monster_damage = monster.damage * (np.log10(monster.health/monster.init_health*100)-1)
        self.health -= np.max([monster_damage,0])
        self.xp += 1
        
    def run(self, monster):
        self.endurance -= monster.endurance/2
        if self.endurance < 0:
            self.health += self.endurance/2
            self.endurance = 0
        self.xp+=1
            
    def take_potion(self, potion):
        self.luck += potion.luck
        self.health += potion.health
        self.damage += potion.damage
        self.endurance += potion.endurance
        if self.endurance <0:
            self.health += self.endurance/2
            self.endurance = 0
    
    def befriend(self, monster):
        luck = self.luck * 0.2
        if np.random.randint(100) < luck:
            print("You befriended the monster")
            return True
        else:
            print("Uh oh! The monster was not friendly. They struck and you were forced to fight")
            self.health -= monster.damage
            return False
        
    def info(self):
        prints("\tHealth:", self.health, "\n\tEndurance:", self.endurance, 
               "\n\tDamange:", self.damage, "\n\tLuck:", self.luck)

## Monster definition

In [6]:
class monster:
    def __init__(self):
        while True:
            values = np.random.randint(5, 41, 3)
            if sum(values) > 35 and sum(values) < 52:
                break
        keys = np.random.choice(monster_attr, 3, replace=False)
        attrs = (dict(zip(keys, values)))
        for key, value in attrs.items():
            setattr(self, key, value)
        self.name = np.random.choice(monster_names)
        self.image = io.imread(path + np.random.choice(monster_pics))
        self.init_health = self.health
    
    def info(self):
        prints("\tHealth:", self.health, "\n\tEndurance:", self.endurance, 
               "\n\tDamange:", self.damage)
        
    def fight(self, player):
        player_damage = player.damage * (np.log10(player.health/player.init_health*100)-1)
        self.health -= np.max([player_damage,0])
        
    def show(self):
        plt.axis('off')
        plt.imshow(self.image);
        plt.show()
    
    def show_dead(self):
        plt.axis('off')
        plt.imshow(np.rot90(self.image[:,::3]//2))
        plt.show()

## Potion definition

In [7]:
class potion:
    def __init__(self, is_trap=False):
        self.name = np.random.choice(list(potions.keys()))
        mul = -0.6 if is_trap else 1
        self.colour = potions[self.name]['colour']
        self.health = potions[self.name]['health'] * mul * rand()
        self.damage = potions[self.name]['damage'] * mul * rand()
        self.luck = potions[self.name]['luck'] * mul * rand()
        self.endurance = potions[self.name]['endurance'] * mul * rand()
        self.image = io.imread(path + self.colour + '_potion.jpg')
    
    def show(self):
        plt.axis('off')
        plt.imshow(self.image);
        plt.show()

## Game definition

In [8]:
def face_monster(player):
    opponent = monster()
    opponent.show()
    prints("You are faced with", opponent.name, "\nThe monster's attributes are:")
    opponent.info()
    prints("what do you want to do?\n1. Fight\n2. Run\n3. Befriend")
    choice = inputs(player, "Select an option: ")
    if choice == "3":
        print("Usually there's only a 10% chance of a monster being friendly.")
        new_choice = inputs(player, "Are you sure?(y/n)")
        if new_choice.lower() == 'y':
            result = player.befriend(opponent)
            if not result:
                choice = "1"
        else:
            choice = inputs(player, "what do you want to do? 1. Fight 2. Run")
    if choice == "2":
        if opponent.endurance > player.endurance*2:
            prints("You could not endure the run and where forced to fight")
            choice = "1"
        else:
            player.score += opponent.endurance
            player.run(opponent)
            prints("you successfully evaded the moster. Your endurance is", player.endurance)
    if choice == "1":
        score = opponent.health
        while opponent.health > 0:
            player.fight(opponent)
            opponent.fight(player)
            if opponent.health > 0:
                prints("Monster is still alive with health", opponent.health, ". Your health is ", player.health)
            if player.health <= 0:
                break
            else:
                opponent.show_dead()
                prints("Monster is dead. Your health is", player.health)
                player.score += score
                break
    return player


def find_potion(player):
    is_trap = check_trap(player.luck)
    potion1 = potion(is_trap)
    potion1.show()
    potion_choice = inputs(player, "you got a %s potion. Do you want to take it? (y/n)" % potion1.name)
    if potion_choice.lower() == 'y':
        player.take_potion(potion1)
        if is_trap:
            prints("Uh oh... it was a trap. Poor luck")
        else:
            prints("You got a bonus")
        prints("Your current status is:")
        player.info()
    else:
        print("You discarded the potion")
    return player


def use_xp(player):
    attribs = ['Health', 'Endurance', 'Luck', 'Damage']
    while player.xp > 0:
        prints("You earned 5 xp. How do you want to use this?\n\t1. %s \n\t2. %s \n\t3. %s \n\t4. %s" % (
                attribs[0], attribs[1], attribs[2], attribs[3]))
        prints("Each xp grants 5x skills")
        xp_choice = inputs(player, "Enter your choice: ")
        if xp_choice == 'stop':
            return quit(player)
        if xp_choice == '':
            xp_choice = None
        else:
            xp_choice = int(xp_choice)
        xp = player.xp * 5
        
        if xp_choice >= 1 and xp_choice <= 4:
            cur_attr_val = getattr(player, attribs[xp_choice-1].lower())
            setattr(player, attribs[xp_choice-1].lower(), cur_attr_val + xp)
        else:
            prints("incorrect choice")
            continue
        player.xp = 0
        print("Your current status is:")
        player.info()
    return player
               

def start_game():
    """Start the game. To quit, type 'stop' at any of the prompts.
    To know your current status, type 'info' at any of the prompts."""
    
    name = input("Name your character: ")
    player = character(name)
    prints("Your attributes are:\n\tEndurance: %s\n\tHealth: %s\n\tLuck: %s\n\tDamage: %s" % (
        player.endurance, player.health, player.luck, player.damage))
    
    while player.health > 0:
        uncertainty = np.random.choice(['monster', 'potion', 'monster', 'potion', 'monster', 'monster'])
        if uncertainty == 'monster':
            player = face_monster(player)
            
        elif uncertainty == 'potion':
            player = find_potion(player)
            
        prints("Moving on...")
        clear_output()
        
        if player.xp >= 5:
            player = use_xp(player)
        time.sleep(1)
        
        if player.quit:
            break
    if player.quit:
        return("You quit with a scrore of %s" %player.score)
    else:
        return("You died with a scrore of %s" %player.score)
    

In [14]:
start_game()

'You quit with a scrore of 0'